# Test Schema View
Notebook that tests how to use the linkml shema view utility for interrogating the NMDC Schema.

In [70]:
from linkml_runtime.utils.schemaview import SchemaView

In [71]:
view = SchemaView('../src/schema/nmdc.yaml')

In [72]:
view.imports_closure()

['core',
 'annotation',
 'attribute_values',
 'external_identifiers',
 'linkml:types',
 'mixs',
 'basic_classes',
 'nmdc_subsets',
 'nmdc_types',
 'basic_slots',
 'portal_enums',
 'portal_emsl',
 'portal_jgi_metagenomics',
 'portal_jgi_metatranscriptomics',
 'portal_mixs_inspired',
 'portal_sample_id',
 'workflow_execution_activity',
 'NMDC']

### Get list of classes (limit to 5)

In [73]:
list(view.all_classes())[0:5]

['ChemicalConversionProcess',
 'MetagenomeAnnotation',
 'FieldResearchSite',
 'Biosample',
 'MobilePhaseSegment']

### Get list of all slots (limit to 5)

In [74]:
list(view.all_slots())[0:5]

['chemical_conversion_category',
 'substances_volume',
 'biosample_categories',
 'collected_from',
 'bulk_elect_conductivity']

### Check if slot is mulitvalued

In [75]:
view.get_slot('study_set').multivalued

### Determine range of slot

In [76]:
view.get_slot('has_input').range

'NamedThing'

### Determine range as specified in the slot_usage

In [77]:
view.get_class('MaterialProcessing').slot_usage['has_input'].range

### If slot is not in slot_usage, an error will be throw. So, use function to determine slot range.

In [79]:
def get_class_slot_range(view_obj, class_name, slot_name):
    if slot_name in view_obj.get_class(class_name).slot_usage:
        return view_obj.get_class(class_name).slot_usage[slot_name].range
    else:
        return view_obj.get_slot(slot_name).range

print(get_class_slot_range(view, 'MaterialProcessing', 'has_input'))
        

None


### Test for non-existent slots

In [ ]:
view.get_slot('foo') ## nothing returned

In [ ]:
'foo' in view.get_class('MaterialProcessing').slots

### Use induced_slot method to find range

In [ ]:
help(SchemaView.induced_slot)

In [ ]:
view.induced_slot('has_input', 'MaterialProcessing').range

In [ ]:
view.induced_slot('has_input', 'DataGeneration').range

### SchemaView won't return the name of the class as it is named in the module. This does.

In [ ]:
import inspect
from nmdc_schema import nmdc


In [ ]:
python_name_dict = {}

for name, member in inspect.getmembers(nmdc):
    if inspect.isclass(member) and hasattr(member, 'class_name'):
        python_name_dict[name] = member.class_name
        
print(list(python_name_dict.items())[0:5])

### Use dict of class names to get info

In [ ]:
# class_name = 'Study'
class_name = 'nmdc:Study' # test for curie
x_name = class_name if -1 == class_name.find(":") else class_name.partition(":")[-1]
x = view.get_class(python_name_dict[x_name])

In [ ]:
x.name

In [ ]:
x.slots

In [ ]:
s = view.get_slot('websites')

In [ ]:
s.description

In [ ]:
list(x.slot_usage) # slot usage is a dict

In [ ]:
s = view.induced_slot('associated_dois', x.name)

In [ ]:
s.description # <-- should have the description: The dataset citation for this study

### If necessary, here is how you retrieve a class using a string.

In [ ]:
eval('nmdc.Biosample')

In [ ]:
eval('Biosample')

In [ ]:
inspect.isclass(eval('Biosample'))

In [ ]:
x = eval('MaterialProcessing')

In [ ]:
x.class_name

### Example of how to get the permissible values for enums

In [ ]:
file_enums = view.get_enum('FileTypeEnum').permissible_values # returns a dict

#### get a permissible value

In [ ]:
tigr = file_enums['TIGRFam Annotation GFF']
tigr

In [ ]:
tigr.description

#### get the annotations for the permissible value
note: trying to get the value from an annotation that doesn't exist will throw an error

In [ ]:
tigr.annotations

In [ ]:
tigr.annotations.get('file_name_pattern', None)

In [ ]:
tigr.annotations.get('file_name_pattern', None).value